# Pygeodyn Explained

## README
Pygeodyn is a python-based wrapper for the GEODYN Orbit Determination Software.  This notebook attempts to explain how to use the the program in a jupyter notebook and detail some of pygeodyn's various parts and functionalities.

Pygeodyn is split up into a few distinct parts:

**The Pygeodyn Program**
 -  `/data/geodyn_proj/pygeodyn/pygeodyn_develop/`
    - `Pygeodyn.py`
       - Serving as the code's entry point, this program is inherits the remaining codes (structured as python Classes) in this list into a module (actually its a python object) to be interacted with by the user.  This object also reads the .yaml settings file that the user inputs.
    - `PYGEODYN_Control.py`
        - This class is inherited into the `Pygeodyn` object and is used by the user to run GEODYN.  It contains all of the functions needed to input geodyn specific files, setup and copy files to a temporary directory, run the geodyn IIS and IIE executables from fortran in the temporary directory, and finally setup and save requested outputs to an output directory. 
    - `PYGEODYN_Read.py`
        - This class is inherited into the `Pygeodyn` object and is used by the user to interpret and view the GEODYN output.  The requested data is identified in the .yaml settings file and then interpreted and saved to pandas DataFrames accordingly. 
        - When the GEODYN run is saved by the Control, the various files (iieout, OrbFil, etc.) are left in their original format, and the Reader is used to interpret this in a user friendly way.
    - `/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/`
        - The codes in the `util_dir` help with the general running and functioning of Pygeodyn.
        - `common_functions.py`
        - `util_classtools.py`
        - `EditSetupFile.py`
    - `/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_preprocessing/`
        - We have added some limited capabilities in processing the G2B files for GEODYN.  So far this has been used with success to construct G2B PCE data for the ICESat2 Satellite."
        -`PYGEODYN_Preprocess.py`

  - **Inclusion of different Satellites**
      - Different satellites and their various inputs/formats are included in Pygeodyn in two ways:
          1. The various datasets and input files are placed in an input directory (`/data/data_geodyn/inputs/icesat2`)
          2. `PYGEODYN_ICESat2.py`: A pygeodyn class in constructed to overwrite any functions in PYGEODYN_Control and PYGEODYN_Read that may need to be specialized for the satellite and its tracking datatype.


    
**The GEODYN source code**  
   - `/data/geodyn_proj/geodyn_code/`
    
   - **IIS**: `/data/geodyn_proj/geodyn_code/IIS`
     - unchanged from original version
   - **IIE**: /data/geodyn_proj/geodyn_code/IIE
       - `ORIG/`
           - The MSIS routine and its references have been renamed to MSIS86.
       - `Kamodo_pygeodyn_MODS/`
          - modified version of the iie subroutines that were used to add msis2, msis00, the interface to Kamodo's Physics models as well as the various modifications necessary to conduct satellite drag research.
       - `CD_model_proj/`
           - Contains the more up-to-date modifications from the `Kamodo_pygeodyn_MODS/` folder and additionally added JB2008, DTM2020, and a physics based CD calculation through the DRIA routine.

**Kamodo** (CCMC's Ionosphere-Thermosphere Model API)
 - `/data/geodyn_proj/interface_kamodo_geodyn`
 - Kamodo Flythrough:
     - `/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough`
     - The code that is used to call Kamodo is nested in `PygeodynControl`, but it calls the `ModelFlythrough()` function in `SatelliteFlythrough.py`.
     - The Kamodo API and its model outputs are interfaced to GEODYN using an OrbitCloud Cube interpolation.



## Running GEODYN

### Overall Steps for Running
   1. Construct a YAML file that contains the `Run_Settings`
   2. Call an instance of the `Pygeodyn` Object that is loaded with the run settings
   3. Use the `Pygeodyn.RUN_GEODYN()` method to run GEODYN with the requested settings

In [1]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

1. Construct a YAML file that contains the `Run_Settings`

The below is an example of the text that should be included in such a file.


In [2]:
'''
---
###  Run Settings Test File 
#
#### Declare paths and naming conventions:
user                         : 'Zach Waldron'
path_to_output_directory     : '/data/zach_work/output_from_runs'
directory_name_specifier     : ''  
PASS_INPUT_VALUE_TO_fortran  :  'None'

arc_length                   : '24hr'
model_data_path              :  None

#### CD MODEL
cd_model  :   'BWDRAG' # 
file_string                  : 'bwdrag'


#### DRIA CD Model Parameters
cd_model_params : { 
                    MS     : None  ,  #!  molar mass for each panel (g/mol)
                    TW     : None  ,  #!  temperature of panels  (K)
                    ALPHA  : None  ,  #!  accomodation coefficient
                    KL     : None  ,  #!  langmuir parameter
                    FRACOX : None  ,  #!  fraction of surface covered by atomic oxygen
                   }

##### CHOOSE GEODYN VERSION:
GEODYN_iie_MOD_version     : 'CD_model_proj'
recompile_on               :  False

### IDENTIFY ARCS   (YYYY.DDD)
arc                 : ['2018.313',]
epoch_start         : ['181109 000000.0000000', ]
epoch_end           : ['181110 000000.0000000', ]


satellite    : 'icesat2' 
den_model    : 'msis2'  
request_data :  ['Trajectory_orbfil',
                'Density',
                'DragFile',
                'Residuals_summary', 
                #'RunSummary',
                # Residuals_obs',
                # 'AdjustedParams',
                                  ]
                                  
                                 
### SETUP FILE OPTIONS 
geodyn_StepSize           : 10.
cd_adjustment_boolean     : False
total_hours_in_run        : None
hours_between_cd_adj      : None
accels                    :  False
empirical_accels          :  False, 
ACCELS                    : 'acceloff'
cards_to_remove           : [ 'ACCEL9',
                              'ORBTVU',
                              'RESID',
                              'CON9PA',]
                  
#### USER AUXILLIARY OPTIONS
verbose         :  False
DRHODZ_update   :  True     #### Do you want to use the updated DRHODZ calculation? (recommended)



#### ----------------------------------------------------------------------------
#### Options that are hardcoded defaults for ICESAT2 class
#### ----------------------------------------------------------------------------
#### The name of the satellite
SATELLITE_dir  : 'icesat2'
#### Satellites COSPAR ID number
SATID          : '1807001'
#### Datatype being used as tracking data
DATA_TYPE      : 'PCE'
#### ----------------------------------------------------------------------------
####### File Choices:
#### G2B file name
g2b_file       : 'g2b_pce_fullset_nomaneuver'  
#### Atmospheric Gravity file name
atgrav_file    : 'ATGRAV.glo-3HR_20160101-PRESENT_9999_AOD1B_0006.0090'
#### Planetary Ephemeris file name
ephem_file     : 'ephem1430.data_2025'
#### Gravity field file name
gravfield_file : 'eigen-6c.gfc_20080101_do_200_fix.grv'
#### PCE Ascii textfile name
StateVector_epochs_datafile  : '/data/data_geodyn/inputs/icesat2/setups/PCE_ascii.txt'



'''
print()


In [3]:
run_settings = '/data/geodyn_proj/GETTING_STARTED/example_run_settings.yaml'

2. Call an instance of the `Pygeodyn` Object that is loaded with the run settings


In [4]:
### Load the data into an object
RunObject = Pygeodyn(run_settings)

3. Use the `Pygeodyn.RUN_GEODYN()` method to run GEODYN with the requested settings

In [5]:
RunObject.RUN_GEODYN()

Run # 1     Current Time =      13:04:58  GMT-7
Run # 1
+ ————————————————————————————————————————————————————————————————————————————————————————
|
| ---------------------- RUN PARAMETERS  ----------------------
|
|  Run # 1     IISSET Cleaned      tmp/.../cleaned_setup_2018292
|  Run # 1     Density Model:      msis2
|  Run # 1     GEODYN Version:     CD_model_proj
|  Run # 1     ARC run:            icesat2_2018292_24hr.msis2.bwdrag
|  Run # 1     Output directory:   /data/zach_work/output_from_runs/msis2/msis2_BWDRAG
|  Run # 1     EXAT File:          /data/data_geodyn/inputs/icesat2/external_attitude/EXAT01.2018.292.gz
|
|  Run # 1     Epoch Start:  2018-10-19 00:00:00
|  Run # 1     Epoch End:    2018-10-20 00:00:00
|  Run # 1     Step Size:    10.0
+ ————————————————————————————————————————————————————————————————————————————————————————

Run # 1          Running IIS
ERRORS FOUND IN IIS: /data/data_geodyn/tmp/msis2_BWDRAG/icesat2_2018292_24hr.msis2.bwdrag/iiserr
STOP 69



SystemExit: 

/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## What is happening behind the scenes?

1. A Pygeodyn object is initialized with the specified run settings saved to a YAML
    - `/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN.py`
        1. The `RunObject` is loaded such that it inherits a Satellite Class(either ICESat2 or Starlette) settings.  This means that the methods (functions attributed to an class) that have been written into the ICESat2 Class will overwrite the functions in the base classes.  The base classes are `PygeodynController` and `PygeodynReader` and they are each inherited into the satellite classes.
        2. So after the `Object = Pygeodyn(run_settings)` call, the following happens:
            - The Pygeodyn class inherits the `Inherit_Icesat2` class which tells Pygeodyn to adopt the functions in the `Satellite_ICESat2` class (`/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_ICESat2.py`).
            - The `Satellite_ICESat2` class inherits both `PygeodynController` and `PygeodynReader`.  With this step, the object initialized by calling `Object = Pygeodyn(run_settings)` now contains all of the functions in `PygeodynController` and `PygeodynReader` and additionally, any functions that are written into `Satellite_ICESat2` with identical names overwrite those in the Controller and Reader.
            
 - Note--- each time a class is called (in the case of `Pygeodyn`) or inherited (the case for `Satellite_ICESat2` and then `PygeodynController` and `PygeodynReader`) the `def __init__()` function runs.  It is in this location that we put anything we want to have happen for each instance of the Class (like any class specific global variables).  If you don't want anything to happen, you can put a pass.

    ```python
    class example_class():
        def __init__(self):  
            pass
    ```




2. Call the `RUN_GEODYN()` method which is now attributed to the object (`RunObject`) that is initialized with the settings and classes as defined above.

    - Explicitly stated again: `RunObject` is an object of the `Pygeodyn` class that inherited the class `Satellite_ICESat2`, which itself inherited `PygeodynController` and `PygeodynReader`).  Therefore `RunObject` contains all of the methods (function for a class) contained in all of those classes.
    -  `RUN_GEODYN()` is called thru these classes and its method is located at the bottom of the PYGEODYN_Control file: `/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_Control.py` 




